# Building a Spam Filter with Naive Bayes

![Cover](images/cover_image.png)

## Introduction

The goal of this project is to build a tool to classify incoming text messages as spam or not spam. We will do this by leveraging the power of conditional probability; in particular, we will implement the multinomial naive Bayes algorithm from the ground up to determine the probability of a message being spam. We will also use Laplace smoothing to avoid getting zero probabilities. 

To train and test the algorithm we will use a dataset of 5,572 SMS that have already been classified by humans. This dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo and is publicly hosted by the [UCI Machine Learning repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). On that page the reader can find more information about the criteria used to collect this corpus of texts together with references to some of the authors' papers where this data has been used.

## Quick Overview of the Naive Bayes Classifier

Before looking at the texts contained in the dataset and starting to build the actual algorithm, for sake of clarity we want to give a quick overview of how the classifier works.

Say that we are given an SMS consisting of $n$ (non-unique) words; denote these words by $w_1,\dots,w_n$. Our goal is to decide whether this SMS is spam or not (from now on, we will use the term *ham* to refer to a SMS which is not spam). To make this decision, we calculate two numbers: the probability the message is spam (which we denote by $P(Spam|w_1, \dots, w_n)$) and the probability the message is ham (which we denote by $P(Ham|w_1, \dots, w_n)$). The formulas to compute these probabilities are the following:
$$
    P(Spam|w_1, \dots, w_n) \propto P(Spam)\prod_{i=1}^n P(w_i|Spam)\\
    P(Ham|w_1, \dots, w_n) \propto P(Ham)\prod_{i=1}^n P(w_i|Ham).
    $$
Here $P(Spam)$ (resp., $P(Ham)$) denotes the probability of receiving a spam message (resp., a ham message); $P(w_i|Spam)$ (resp., $P(w_i|Ham)$) denotes the probability that a spam message (resp., a ham message) contains the word $w_i$; the symbol $\propto$ means *is proportional to* and, for all concrete purposes, in this context it can be interpreted as an equal sign.

The important thing to keep in mind is that all these probabilities can be computed using the already classified SMS in the dataset through ad-hoc formulas; we will cover these formulas in details later on once we start implementing the algorithm. For now, to give some extra context, I will just say the equations that appear above are a consequence of [Bayes theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) (which is directly related to the notion of [*conditional probability*](https://en.wikipedia.org/wiki/Conditional_probability)) used under the assumption of [conditional independence](https://en.wikipedia.org/wiki/Conditional_independence#Conditional_independence_of_events) between the words appearing in a message (this last hypothesis is rather simplistic and is what earns this classifier the attribute *naive*).

---

## A Brief Exploration of the Texts Collection

Now that we have a better idea of how the naive Bayes algorithm will work, let's focus on the SMS we have available.

In [1]:
# Makes Jupyter print all variables appearing in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

# Read in the dataset and manually define the header
sms_collection = pd.read_csv("SMSSpamCollection", sep = "\t", header = None, names = ["Label", "SMS"])

In [3]:
# Show first five rows
sms_collection.head()

print("There are {} texts in the collection.".format(sms_collection.shape[0]))

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


There are 5572 texts in the collection.


The dataframe has only two columns: the first one categorizes the SMS as spam or ham and the second one contains the entire text of the message. We also get confimation that the dataset contains 5,572 SMS.  

In [4]:
# Check percentage of ham/spam messages
sms_collection["Label"].value_counts(normalize = True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The majority of texts (more than 86%) in the dataset are ham. This in line with a real life situation where the spam messages represent only a small percentage of all texts a person receives.

## Splitting the Dataset into Training Set and Test Set

Before starting to build our spam filter, we split the SMS collection into two sets:
- The first one represents the **training set**. We use these texts to calculate the probabilities that are required in the algorithm. We decide to use 80% of the total number of SMS for this purpose.
- The second one represents the **test set**. We will use these texts at the end of the project, after we have built the spam filter, to test its efficiency. We use the remaining 20% of th SMS for this purpose.

Before splitting the dataset using this proportions, we first randomize it to make sure spam and ham messages are well mixed up.

In [5]:
# Randomize the dataset
shuffled_sms = sms_collection.sample(frac = 1, random_state = 1) # Specify random_state for reproducibility

In [6]:
# Create training set
training_set = shuffled_sms.sample(frac = 0.8, random_state = 1)

# Create test set 
test_set = shuffled_sms.sample(frac = 0.2, random_state = 1)

# Reset indexes
training_set.reset_index(drop = True, inplace = True)
test_set.reset_index(drop = True, inplace = True)

In [7]:
print("The training set contains {} SMS.".format(training_set.shape[0]))
print("The percentages of ham/spam messages are:")
training_set["Label"].value_counts(normalize = True)*100

print("-"*10)

print("The test set contains {} SMS.".format(test_set.shape[0]))
print("The percentages of ham/spam messages are:")
test_set["Label"].value_counts(normalize = True)*100

The training set contains 4458 SMS.
The percentages of ham/spam messages are:


ham     86.675639
spam    13.324361
Name: Label, dtype: float64

----------
The test set contains 1114 SMS.
The percentages of ham/spam messages are:


ham     86.175943
spam    13.824057
Name: Label, dtype: float64

We end up with 4,458 messages (out of the 5,572 total) in the training set and the remaining 1,114 in the test set.

It should also be noted how the percentages of ham and spam messages within each set reflect very accurately the percentages in the full dataset.

## Cleaning Up the Text of the Messages

Now it's time to start designing the actual spam filter. We have already explained how, in order to implement the naive Bayes algorithm, given a word $w$, it is crucial to calculate the probabilities $P(w|Spam)$ and $P(w|Ham)$. For this, we will need to count the number of times $w$ appears in spam or ham messages contained in the training set.   

To allow these calculations, we first prepare the training set by doing some text cleaning operations. In particular, in what follows we make each text lowercase and, since they don't usually tell much about the integrity of a message, we also eliminate any punctuation mark .

In [8]:
training_set

,Label,SMS
0,ham,Good night my dear.. Sleepwell&amp;Take care
1,ham,Sen told that he is going to join his uncle fi...
2,ham,Thank you baby! I cant wait to taste the real ...
3,ham,When can ü come out?
4,ham,No. Thank you. You've been wonderful
...,...,...
4453,ham,Sun cant come to earth but send luv as rays. c...
4454,ham,Well boy am I glad G wasted all night at apple...
4455,ham,Ya:)going for restaurant..
4456,ham,Anyway seriously hit me up when you're back be...


In [9]:
# Make text lowercase
training_set["SMS"] = training_set["SMS"].str.lower()

In [10]:
# Remove punctuation marks
training_set["SMS"] = training_set["SMS"].str.replace(r"\W{2,}", " ", regex = True) # Makes sure not to leave
                                                                                    # extra white space
training_set["SMS"] = training_set["SMS"].str.replace(r"\W", " ", regex = True)

In [11]:
# Check the result
training_set

,Label,SMS
0,ham,good night my dear sleepwell amp take care
1,ham,sen told that he is going to join his uncle fi...
2,ham,thank you baby i cant wait to taste the real t...
3,ham,when can ü come out
4,ham,no thank you you ve been wonderful
...,...,...
4453,ham,sun cant come to earth but send luv as rays cl...
4454,ham,well boy am i glad g wasted all night at apple...
4455,ham,ya going for restaurant
4456,ham,anyway seriously hit me up when you re back be...


## Determining the Training Set Vocabulary

The next step is to create a list of all the words we want to consider. This is what is called the *vocabulary* and corresponds to all the words appearing in SMS contained in the training set. We will use this list in the next step, before actually implementing the algorithm, to organize the training set in a more convenient way.   

In [12]:
# Create a list of lists each of them containing all words in a message
sms_list = training_set["SMS"].str.split().tolist()

# Remove one level of nesting to create the list of words in the vocabulary
non_unique_vocabulary = [word for sms in sms_list for word in sms]

# Remove duplicate words
vocabulary = list(set(non_unique_vocabulary))

print("The vocabulary of the training set consists of {} words.".format(len(vocabulary)))

The vocabulary of the training set consists of 7712 words.


## Converting the Training Set into a More Suitable Format

As we have already hinted, to calculate the probability that a message is spam or ham, we need to look at how many times each available word (i.e., each word in the vocabulary) appears in each category of message (spam or ham). To perform these calculations more quickly, it is a good idea to first turn the training set into a more convenient format. 

In concrete, we will create a new column for each word in the vocabulary: the value of each row at such column will then show how many times that word appears in the corresponding message.

We create these columns using a dictionary: we use as keys the words in the vocabulary and we update the value corresponding to each key by analyzing the messages in the training set one at a time.

In [13]:
# Initialize dictionary
word_counts_per_sms = {word : [0]*len(training_set["SMS"])
    for word in vocabulary}

In [14]:
# Update values in dictionary by looping through the messages and its words
for index, message in enumerate(training_set["SMS"]):
    for word in message.split():
        word_counts_per_sms[word][index] += 1 

Let's just do a quick check to make sure there's no column, among those we created, that is entirely filled with zeros (such a scenario would imply that some word in the vocabulary was not found in any message, which would imply something went wrong). 

In [15]:
problematic_words = 0

# Count the number of words in the vocabulary which did not appear in any message (we want this count to be 0)
for word in vocabulary:
    if sum(word_counts_per_sms[word]) == 0:
        problematic_words += 1

In [16]:
if problematic_words > 0:
    print("There are {} word in the vocabulary that were not found in any message: this is a problem!".format(
        problematic_words))
else:
    print("There's no column entirely filled with zeros. The columns you created are good!")

There's no column entirely filled with zeros. The columns you created are good!


Now that we are sure all the columns we created look fine, we organize them into a single dataframe.

In [17]:
# Create dataframe with word counts
word_counts_per_sms_df = pd.DataFrame(word_counts_per_sms)

In [18]:
# Make Jupyter display all columns in a DataFrame
pd.set_option('display.max_columns', 999)

# Look at the result
word_counts_per_sms_df  # By design most of the entries will be zeros

,yaxxx,gone,premier,tnc,sem,2moro,trouser,incident,mates,read,suite342,08081560665,blog,main,08715203677,rally,that2worzels,sweetest,negative,capital,loads,preponed,before,position,c52,pract,hilarious,gam,stolen,blessings,shu,road,irulinae,oooooh,voda,tmrw,aunty,howdy,guilty,ordered,dose,luvs,digi,walking,aight,train,08707500020,sometime,nichols,extract,upto,pilates,canceled,licks,bbq,lemme,900,iraq,batt,tats,admin,necesity,convincing,008704050406,èn,understood,zealand,bedroom,satanic,dress,kodstini,weddingfriend,suddenly,08001950382,durban,srt,american,england,tea,including,diamonds,juicy,medical,parade,rd,size,mornin,mnths,mymoby,must,remixed,fortune,merry,belligerent,thing,trauma,sucker,pray,after,abj,literally,paracetamol,ything,sinco,india,callcost,arts,suffers,fan,wahleykkum,same,un,pride,mathe,tok,magic,suggestion,530,minmoremobsemspobox45po139wa,manky,appreciated,netflix,liquor,clark,net,scallies,possessiveness,crore,profit,wasting,payoh,cancer,ne,didnt,supose,csbcm4235wc1n3xx,notifications,ordinator,aproach,um,passable,grinder,ex,07821230901,studdying,happening,horrible,synced,cancel,sextextuk,points,skateboarding,lush,hussey,fujitsu,motherfucker,wildest,tonght,uploaded,bone,pure,pulls,glory,paru,nokia,4info,roses,recently,xin,goodnight,tunji,dislikes,box403,hmmm,dammit,quoting,navigate,sutra,prescripiton,fundamentals,drop,keen,malarky,tactful,bad,smokin,brats,otbox,empty,lot,forever,algarve,08715203685,ttyl,comp,saves,returns,87121,snowman,786,jan,manchester,unbelievable,don,rayman,appointments,dental,ass,clearly,build,havin,spiritual,rajnikant,starving,recycling,millions,girls,yellow,destination,chatlines,bob,random,slow,bothering,nasdaq,misundrstud,despite,peripherals,ans,action,glass,suprman,hmv,usual,phil,mentioned,exhaust,celebrations,apologise,immediately,weight,31p,senor,88066,hotel,kane,inviting,dhina,stopped,expected,sachin,beta,town,public,jaya,lonlines,tv,loans,prospects,ktv,flirt,clue,shock,londn,perhaps,knee,tscs087147403231winawk,28days,heat,ah,regret,settle,8077,earlier,sehwag,ready,honest,walk,20p,age16,soil,goodfriend,reverse,weapon,n,risks,red,signing,bros,faded,textcomp,thinkthis,flood,mw,frnd,2docd,attach,st,mobile,tour,golden,hairdressers,miracle,best,dun,unnecessarily,4give,stores,bubbletext,vikky,07099833605,09090900040,cnupdates,hcl,locaxx,sabarish,beware,hv9d,pocay,homeowners,nite,yck,woo,simulate,latest,travelled,polo,drinking,smsing,cheers,allah,vomiting,sorry,2find,nimbomsons,rencontre,leaving,hoops,lazy,stalk,gravity,12hrs,dogging,kudi,sec,iouri,woke,4mths,freek,phonebook,drastic,s,apnt,secondary,wants,woohoo,chinese,grumble,doesdiscount,hack,pass,enjoyin,forwarded,picture,09061790121,custom,blessing,more,curry,claire,9,bit,your,individual,screen,tm,ibm,haul,erupt,elsewhere,ldew,shijutta,quit,walmart,avoid,jontin,noworriesloans,heltini,possibility,weigh,renewed,noun,jordan,09064012103,zogtorius,silently,45239,dining,shocking,lesson,na,1da,hui,3,rush,timi,semester,86888,wear,relatives,scared,box97n7qp,asp,conveying,sha,ultimate,pert,unsub,2gthr,travel,eatin,swoop,wow,coffee,html,cheyyamo,privacy,physics,booking,others,spoke,uni,kotees,cres,ors,81618,bettr,pushbutton,pound,400thousad,nobbing,everyday,butt,visionsms,077xxx,gave,booked,2007,length,ill,colleagues,twelve,british,boo,dodgey,avent,australia,gbp4,only,sufficient,bleh,evr,viva,moon,janinexx,particularly,priest,87021,pours,dent,itcould,sat,asssssholeeee,whenevr,113,gotmarried,feed,ternal,kidz,thy,shipped,spider,vegas,skyped,handset,if,1843,hour,natural,wings,sent,88088,fans,trained,active,10,astronomer,weiyi,potential,...,rubber,apples,09064019014,6,law,pic,oz,08000776320,mum,grandma,fails,tirunelvai,i,depressed,children,hor,fly,fffff,marvel,parantella,beads,mentionned,cold,olayiwola,counts,perform,shipping,gaze,luv,poker,88600,statement,oja,deus,74355,shinco,drvgsto,atlast,é,eurodisinc,liver,midnight,lautech,xavier,basket,lanre,unintentionally,audition,four,cabin,submitting,smashed,dedicate,combination,kaaj,nearly,search,forgets,me

Finally, we append this dataframe to the training set to keep track of each message and its label. 

In [19]:
# Append this dataframe to the training set
modified_training_set = pd.concat([training_set, word_counts_per_sms_df], axis = "columns")

Let's take a look at the final result.

In [20]:
modified_training_set

,Label,SMS,yaxxx,gone,premier,tnc,sem,2moro,trouser,incident,mates,read,suite342,08081560665,blog,main,08715203677,rally,that2worzels,sweetest,negative,capital,loads,preponed,before,position,c52,pract,hilarious,gam,stolen,blessings,shu,road,irulinae,oooooh,voda,tmrw,aunty,howdy,guilty,ordered,dose,luvs,digi,walking,aight,train,08707500020,sometime,nichols,extract,upto,pilates,canceled,licks,bbq,lemme,900,iraq,batt,tats,admin,necesity,convincing,008704050406,èn,understood,zealand,bedroom,satanic,dress,kodstini,weddingfriend,suddenly,08001950382,durban,srt,american,england,tea,including,diamonds,juicy,medical,parade,rd,size,mornin,mnths,mymoby,must,remixed,fortune,merry,belligerent,thing,trauma,sucker,pray,after,abj,literally,paracetamol,ything,sinco,india,callcost,arts,suffers,fan,wahleykkum,same,un,pride,mathe,tok,magic,suggestion,530,minmoremobsemspobox45po139wa,manky,appreciated,netflix,liquor,clark,net,scallies,possessiveness,crore,profit,wasting,payoh,cancer,ne,didnt,supose,csbcm4235wc1n3xx,notifications,ordinator,aproach,um,passable,grinder,ex,07821230901,studdying,happening,horrible,synced,cancel,sextextuk,points,skateboarding,lush,hussey,fujitsu,motherfucker,wildest,tonght,uploaded,bone,pure,pulls,glory,paru,nokia,4info,roses,recently,xin,goodnight,tunji,dislikes,box403,hmmm,dammit,quoting,navigate,sutra,prescripiton,fundamentals,drop,keen,malarky,tactful,bad,smokin,brats,otbox,empty,lot,forever,algarve,08715203685,ttyl,comp,saves,returns,87121,snowman,786,jan,manchester,unbelievable,don,rayman,appointments,dental,ass,clearly,build,havin,spiritual,rajnikant,starving,recycling,millions,girls,yellow,destination,chatlines,bob,random,slow,bothering,nasdaq,misundrstud,despite,peripherals,ans,action,glass,suprman,hmv,usual,phil,mentioned,exhaust,celebrations,apologise,immediately,weight,31p,senor,88066,hotel,kane,inviting,dhina,stopped,expected,sachin,beta,town,public,jaya,lonlines,tv,loans,prospects,ktv,flirt,clue,shock,londn,perhaps,knee,tscs087147403231winawk,28days,heat,ah,regret,settle,8077,earlier,sehwag,ready,honest,walk,20p,age16,soil,goodfriend,reverse,weapon,n,risks,red,signing,bros,faded,textcomp,thinkthis,flood,mw,frnd,2docd,attach,st,mobile,tour,golden,hairdressers,miracle,best,dun,unnecessarily,4give,stores,bubbletext,vikky,07099833605,09090900040,cnupdates,hcl,locaxx,sabarish,beware,hv9d,pocay,homeowners,nite,yck,woo,simulate,latest,travelled,polo,drinking,smsing,cheers,allah,vomiting,sorry,2find,nimbomsons,rencontre,leaving,hoops,lazy,stalk,gravity,12hrs,dogging,kudi,sec,iouri,woke,4mths,freek,phonebook,drastic,s,apnt,secondary,wants,woohoo,chinese,grumble,doesdiscount,hack,pass,enjoyin,forwarded,picture,09061790121,custom,blessing,more,curry,claire,9,bit,your,individual,screen,tm,ibm,haul,erupt,elsewhere,ldew,shijutta,quit,walmart,avoid,jontin,noworriesloans,heltini,possibility,weigh,renewed,noun,jordan,09064012103,zogtorius,silently,45239,dining,shocking,lesson,na,1da,hui,3,rush,timi,semester,86888,wear,relatives,scared,box97n7qp,asp,conveying,sha,ultimate,pert,unsub,2gthr,travel,eatin,swoop,wow,coffee,html,cheyyamo,privacy,physics,booking,others,spoke,uni,kotees,cres,ors,81618,bettr,pushbutton,pound,400thousad,nobbing,everyday,butt,visionsms,077xxx,gave,booked,2007,length,ill,colleagues,twelve,british,boo,dodgey,avent,australia,gbp4,only,sufficient,bleh,evr,viva,moon,janinexx,particularly,priest,87021,pours,dent,itcould,sat,asssssholeeee,whenevr,113,gotmarried,feed,ternal,kidz,thy,shipped,spider,vegas,skyped,handset,if,1843,hour,natural,wings,sent,88088,fans,trained,active,10,astronomer,...,rubber,apples,09064019014,6,law,pic,oz,08000776320,mum,grandma,fails,tirunelvai,i,depressed,children,hor,fly,fffff,marvel,parantella,beads,mentionned,cold,olayiwola,counts,perform,shipping,gaze,luv,poker,88600,statement,oja,deus,74355,shinco,drvgsto,atlast,é,eurodisinc,liver,midnight,lautech,xavier,basket,lanre,unintentionally,audition,four,cabin,submitting,smashed,dedicate,combination,kaaj,nearly,search,forgets,merely,a

As a reward for this data cleaning and data wrangling job we have done, we now have the training set in a convenient format which will make it easy to calculate all the numbers we need.

## Calculating the Constants for the Algorithm

Time for some explict calculations! In the introduction, we have already seen the formulas used in the naive Bayes algorithm to obtain the probability that a message consisting of words $w_1, \dots, w_n$ is spam or ham are:

$$
    P(Spam|w_1, \dots, w_n) \propto P(Spam)\prod_{i=1}^n P(w_i|Spam)\\
    P(Ham|w_1, \dots, w_n) \propto P(Ham)\prod_{i=1}^n P(w_i|Ham).
    $$
    
We can quickly find the constants $P(Spam)$ and $P(Ham)$ by checking how many of the messages in the training set are spam and how many are ham.  

In [21]:
# Calculate P(Ham)
ham_subset = modified_training_set[modified_training_set["Label"] == "ham"] 
p_ham = len(ham_subset) / len(modified_training_set)

# Calculate P(Spam)
spam_subset = modified_training_set[modified_training_set["Label"] == "spam"] 
p_spam = len(spam_subset) / len(modified_training_set)

In [22]:
print("P(Ham) = ", p_ham)
print("P(Spam) = ", p_spam)

P(Ham) =  0.8667563930013459
P(Spam) =  0.13324360699865412


To calculate the values of $P(w_i|Spam)$ and $P(w_i|Ham)$, we use the following equations:

$$
    P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha N_{Vocabulary}}\\
    P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha N_{Vocabulary}}
    $$

where: 
- $N_{w_i|Spam}$ (resp., $N_{w_i|Ham}$) is the number of times the word $w_i$ occurs within Spam messages (resp., Ham messages);
- $N_{Spam}$ (resp., $N_{Ham}$) is the number of (non-unique) words appearing within Spam messages (resp., Ham messages);
- $N_{Vocabulary}$ is the number of words in the vocabulary of the training set;
- $\alpha$ is a smoothing parameter that we add to avoid getting zero probabilities. In our case we use Laplace smoothing, i.e., we set $\alpha = 1$.

Let's compute the three remaining constants $N_{Spam}, N_{Ham}$ and $N_{Vocabulary}$.

In [23]:
# Calculate N_vocabulary
N_voc = len(vocabulary)

# Calculate N_spam
spam_messages = spam_subset["SMS"]
N_spam = spam_messages.str.split().apply(len).sum()

# Calculate N_ham
ham_messages = ham_subset["SMS"]
N_ham = ham_messages.str.split().apply(len).sum()

In [24]:
print("N_(Vocabulary) = ", N_voc)
print("N_(Ham) = ", N_ham)
print("N_(Spam) = ", N_spam)

N_(Vocabulary) =  7712
N_(Ham) =  57140
N_(Spam) =  15142


Let's also set the value of the smoothing parameter $\alpha$ we mentioned above.

In [25]:
# Initiate alpha
alpha = 1

## Calculating the Parameters for the Algorithm

We still need to calculate the remaining parameters required by the algorithm; in other words, we need to calculate the probabilities $P(w|Spam)$ and $P(w|Ham)$, for each word $w$ in the vocabulary. We have already expliticly given above the formulas we need for this.

Our vocabulary has 7,712 words so we will need to calculate a total of 15,424 probabilities (for each word we need to calculate both $P(w|Spam)$ and $P(w|Ham)$). This sounds like an hefty amount of calculations we are putting the machine through; however, notice how this is the only core computational step since, once we get these numbers, the algorithm can just pick out these probabilities each time, without needing to compute them again for every new message we want to classify. 

This is one of the reasons why the naive Bayes algorithm is considered fast and efficient.

In [26]:
# Function to calculate N_(w|Spam)
def N_w_in_spam(w):
    return spam_subset[w].sum()

# Function to calculate P(w|Spam)
def p_w_given_spam(w):
    num = N_w_in_spam(w) + alpha
    den = N_spam + alpha * N_voc
    return num / den

# Organize all P_(w|Spam) into a dictionary
p_w_spam = {}

for word in vocabulary:
    p_w_spam[word] = p_w_given_spam(word)

# Function to calculate N_(w|Ham)
def N_w_in_ham(w):
    return ham_subset[w].sum()

# Function to calculate P(w|Ham)
def p_w_given_ham(w):
    num = N_w_in_ham(w) + alpha
    den = N_ham + alpha * N_voc
    return num / den

# Organize all P_(w|Ham) into a dictionary
p_w_ham = {}

for word in vocabulary:
    p_w_ham[word] = p_w_given_ham(word)

## Finally, Classifying New Messages

So far, we have cleaned the data in the training set and we have used it to calculate all the constants and parameters needed in the multinomial naive Bayes algorithm. Now, we are finally in the position to start classifying new messages. In particular, we will use the probabilities we computed above to determine the chances that an incoming message is spam or ham: if it is more likely that it is spam, then we will label it as a spam message; if instead is more likely to be ham, then we will classify it as such. If the two possibilites are equally probable (an event which we expect to occur in very few cases) then we will require a human check to determine the outcome.

Note how it is possible that an incoming message contains words that do not belong to the vocabulary of our training set. For these words we cannot easily calculate any metrics so, instead, we will just disregard them (ideally, nearly all words we'll encounter do appear in the training set so this approach won't affect the performance of our classifier).

In [27]:
import re

# Classifying function
def classify(message):
    """
    Given a text message, it runs the multinomial naive Bayes algorithm and it prints both the probability the
    text is ham and the probability the text is spam. According to which of these two is greater, it specifies
    whether the text should be considered "spam" or "ham". In case the two probabilities are exactly equal,
    it prints a message specifying that a human check is required.
    """    
    
    # Clean message
    message = re.sub(r"\W{2,}", " ", message) # removing non-words characters
    message = re.sub(r"\W", " ", message) 
    message = message.lower() # making every word lowercase
    
    message = message.split()
    
    # Initialize probabilities
    p_spam_message = p_spam
    p_ham_message = p_ham
    
    # Update probabilities checking each word
    for word in message:  
        if word in vocabulary:
            p_spam_message *= p_w_spam[word]
            p_ham_message *= p_w_ham[word]
        else:
            pass
        
    print("P(Spam|message) =", p_spam_message)
    print("P(Ham||message) =", p_ham_message)
    
    # Determine correct label
    if p_spam_message > p_ham_message:  
        print("The message is Spam.")
    elif p_ham_message > p_spam_message:
        print("The message is Ham.")
    else:
        print("The two labels are equally probable. A human check is required to classify the message.")

Let's do a first test of our classifying function by feeding it two messages, one from each category.

In [28]:
# Test classification function on a spam message
message = "YOU ARE THE LUCKY WINNER OF TODAY'S SURPRISE RAFFLE. Click here to collect your prize."
classify(message)

P(Spam|message) = 8.560673621331196e-43
P(Ham||message) = 4.82677645171362e-47
The message is Spam.


In [29]:
# Test classification function on a ham message
message = "No, I had an hamburger yesterday. I would rather have a salad tonight."
classify(message)

P(Spam|message) = 4.2865015522911855e-40
P(Ham||message) = 2.510591855140167e-34
The message is Ham.


## Testing the Algorithm

We are at a very good point: we implemented the naive Bayes algorithm by defining a classifying function which determines if a given message is spam or ham; we also ran this function through a small test by feeding it two messages which were, by design, easy to classify. 

Now it's come the time of truth: we will run our classifying function through the test set of 1,114 messages for which we already know the correct spam/ham label. Then, we will be able to tell how many labels our function "guessed right".

First of all we rewrite a modified version of our classifying function which, instead of printing out text, returns the label to be assigned to the message. Then we apply this function to the messages in the test set.

In [30]:
# Classifying function for test set
def classify_and_label(message):
    """
    Runs the multinomial naive Bayes algorithm to classify and label a given text message as "spam" or "ham". 
    In case neither of these two labels is preferable (i.e., if the two probabilities computed by the
    algorithm are exactly equal), then the "needs human check" label is returned.
    
    :params message: the text we want to classify
    :type message: str

    :return: the label which classifies the message. There are three possible values: "spam", "ham"
             or "needs human check"
    """
    
    # Clean message
    message = re.sub(r"\W{2,}", " ", message) # removing non-words characters
    message = re.sub(r"\W", " ", message) 
    message = message.lower() # making every word lowercase
    
    message = message.split()
    
    # Initialize probabilities
    p_spam_message = p_spam
    p_ham_message = p_ham
    
    # Update probabilities checking each word
    for word in message:  
        if word in vocabulary:
            p_spam_message *= p_w_spam[word]
            p_ham_message *= p_w_ham[word]
        else:
            pass
    
    # Determine the appropriate label
    if p_spam_message > p_ham_message:  
        return "spam"
    elif p_ham_message > p_spam_message:
        return "ham"
    else:
        return "needs human check"

In [31]:
# Apply function to test set
test_set["predicted_label"] = test_set["SMS"].apply(classify_and_label)

In [32]:
test_set

,Label,SMS,predicted_label
0,ham,Good night my dear.. Sleepwell&amp;Take care,ham
1,ham,Sen told that he is going to join his uncle fi...,ham
2,ham,Thank you baby! I cant wait to taste the real ...,ham
3,ham,When can ü come out?,ham
4,ham,No. Thank you. You've been wonderful,ham
...,...,...,...
1109,spam,"Dear Voucher Holder, To claim this weeks offer...",spam
1110,ham,So li hai... Me bored now da lecturer repeatin...,ham
1111,ham,Ugh hopefully the asus ppl dont randomly do a ...,ham
1112,ham,Allo! We have braved the buses and taken on th...,ham


What is left to do is measuring the accuracy of our classifying function by checking the percentage of labels that were predicted correctly. We expect to reach an accuracy value of at least 80%. If this number is not met, then we will have to go back and modify our function to make it more accurate.

In [33]:
# Compare labels
prediction_outcome = test_set["Label"] == test_set["predicted_label"]

In [34]:
accuracy = prediction_outcome.sum() / len(prediction_outcome)

print("The classifying function has an accuracy of {}%.".format(round(accuracy * 100, 3)))

The classifying function has an accuracy of 99.282%.


Great! The function we built turns out to be over 99% accurate, which largely exceeds our expectations.

## Assessing the Wrongly Predicted Messages

We are very happy with the accuracy we achieved. Yet, let's isolate those message that were wrongly classified and try to guess what went wrong.

In [35]:
# Makes Jupyter show entire content of dataframe cells
pd.set_option('max_colwidth', None)

# Filter out all wrongly classified messages
filtered_index = prediction_outcome[prediction_outcome == False].index
wrongly_classified = test_set.iloc[filtered_index]

# Filter out "false negative" messages
wrongly_classified_ham = wrongly_classified[wrongly_classified["predicted_label"] == "ham"]
# Filter out "false positive" messages
wrongly_classified_spam = wrongly_classified[wrongly_classified["predicted_label"] == "spam"]
# Filter out "needs human check" messages
needs_human_check = wrongly_classified[wrongly_classified["predicted_label"] == "needs human check"]

In [36]:
print("A total of {} messages were wrongly classified. Among these:".format(len(wrongly_classified)))
print("{} were false negatives (i.e., were classified ham but were actually spam);".format(
    len(wrongly_classified_ham)))
print("{} were false positives (i.e., were classified spam but were actually ham);".format(
    len(wrongly_classified_spam)))
print("{} needs human check (i.e., the ham/spam probabilities were exactly the same).".format(
    len(needs_human_check)))

print("-"*100)
print("Wrongly classified HAM messages:")
wrongly_classified_ham

print("-"*100)
print("Wrongly classified SPAM messages:")
wrongly_classified_spam

print("-"*100)
print("Messages which NEED A HUMAN CHECK:")
needs_human_check

A total of 8 messages were wrongly classified. Among these:
5 were false negatives (i.e., were classified ham but were actually spam);
2 were false positives (i.e., were classified spam but were actually ham);
1 needs human check (i.e., the ham/spam probabilities were exactly the same).
----------------------------------------------------------------------------------------------------
Wrongly classified HAM messages:


,Label,SMS,predicted_label
115,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",ham
415,spam,"Hello darling how are you today? I would love to have a chat, why dont you tell me what you look like and what you are in to sexy?",ham
467,spam,Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net,ham
482,spam,Missed call alert. These numbers called but left no message. 07008009200,ham
613,spam,Would you like to see my XXX pics they are so hot they were nearly banned in the uk!,ham


----------------------------------------------------------------------------------------------------
Wrongly classified SPAM messages:


,Label,SMS,predicted_label
96,ham,Anytime...,spam
413,ham,No calls..messages..missed calls,spam


----------------------------------------------------------------------------------------------------
Messages which NEED A HUMAN CHECK:


,Label,SMS,predicted_label
159,ham,"The last thing i ever wanted to do was hurt you. And i didn't think it would have. You'd laugh, be embarassed, delete the tag and keep going. But as far as i knew, it wasn't even up. The fact that you even felt like i would do it to hurt you shows you really don't know me at all. It was messy wednesday, but it wasn't bad. The problem i have with it is you HAVE the time to clean it, but you choose not to. You skype, you take pictures, you sleep, you want to go out. I don't mind a few things here and there, but when you don't make the bed, when you throw laundry on top of it, when i can't have a friend in the house because i'm embarassed that there's underwear and bras strewn on the bed, pillows on the floor, that's something else. You used to be good about at least making the bed.",needs human check


It looks like our classifier is more biased towards ham texts; in fact, most of the wrongly classified SMS were actually spam. This is in line with the nature of the problem: the vast majority of texts we receive (and the majority of the ones contained in the dataset too) are ham hence it is more probable that a filter lets go through a few (possibly very sneaky) spam messages rather than blocking some legit ones.

We also notice there is a single text the algorithm wasn't able to classify: upon revision, judging from its lenght and how intricate it is, this SMS is most definitely ham.

## Conclusion

In this project we created a spam filter from scratch implementing the multinomial naive Bayes algorithm and taking advantage of a collection of 5,572 already classified SMS. In detail:

1. We gave a quick overview of how the naive Bayes algorithm works and what kind of information it uses;
2. We divided the dataset of text messages into a training and test set;
3. In order to make the algorithm run more smoothly, we cleaned the SMS in the training set making every word lowercase and removing punctuation marks;
4. We found the vocabulary of words that appear in the SMS contained in the training set;
5. We used these words to add columns to the training data and through these new columns we calculated the constants one needs to run the naive Bayes algorithm (we also set a Laplace smoothing parameter $\alpha = 1$ to avoid getting zero probabilities);
6. We computed the remaining parameters and implemented the spam filter by defining a function which takes as input a message and returns its classification label: either spam or ham (i.e., not spam);
7. We applied this function to the test set and observed an accuracy of 99.282%;
8. More specifically, out of the 1,114 SMS contained in the test set, only 8 of them were wrongly classified: 5 of them were wrongly classified ham, 2 were wrongly classified spam and 1 needed human check (because the probabilities of being ham or spam were exactly the same);
9. After taking a look at this ambiguous text, we concluded it is definitely ham. 

## Next Steps

The following is a list of possible ideas that can be expanded in order to improve this project:

- The accuracy score we obtained almost seems too good to be true. To make sure we are not running into overfitting issues, it might be a good idea to test our tool on a completely new setof pre-classified SMS and see if we still obtain an accuracy of at least 80% (the minimum benchmark we were initially aiming for); 
- If this threshold is not met, we can try to improve the algorithm by changing the way we clean the messages. For example, we can make the classifier case sensitive by leaving both upper and lowercase words;
- We can use the classifying function we defined to create a full fledge app which users can interact with by inputting the message that needs to be classified;
- Taking the previous step even further, we can try to automate the classifying function so that it triggers every time a new text is received. My current weak skills as a developer won't probably allow me to achieve this result but I'm sure that, after some readings and consulting the right resources, I would eventually be able to pull it off.